# 6.2. Convolutions for Images

In [0]:
import torch
from torch import nn

- 2D Convolution을 수행하는 corr2d함수를 아래와 같이 구현하겠습니다. 
- 실제 구현된 함수는 2D correlation입니다. Correlation을 통해 어떻게 convolution을 수행할 수 있는지를 이해하도록 합니다.
- Y의 shape가 어떻게 정해진 것인지 이해하도록 합니다.


In [0]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

- 간단한 예제를 통해 구현한 함수가 제대로 동작하는 지를 확인해 봅니다.

In [3]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K = torch.tensor([[0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

- 이제 양 끝의 2개의 column이 1이고, 나머지는 0인 X를 정의하겠습니다.

In [4]:
X = torch.ones((6,8))
X[:, 2:6] = 0
print(X)
print(X.shape)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
torch.Size([6, 8])


- 연산에 사용할 kernel은 [1, -1] 입니다. 어떤 역할을 하는 kernel인지 상상해 봅시다.

In [5]:
K = torch.tensor([[1,-1]])
print(K)
print(K.shape)
#print(K.t())
#print(K.t().shape)

tensor([[ 1, -1]])
torch.Size([1, 2])


- X와 K의 연산결과는 가로 방향으로의 difference임을 우리는 결과를 통해 확인할 수 있습니다. 

In [6]:
Y = corr2d(X, K)
Y
#Yt = corr2d(X.t(),K)
#Yt

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

- 이제 2개의 parameter를 가지는 학습모델을 구성하고, X와 Y를 이용하여 parameter를 찾는 과정을 연습해 보겠습니다.

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size = (1, 2), bias = False)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

print(conv2d.weight.data)
print(conv2d.weight.grad)

tensor([[[[-0.2339, -0.3929]]]])
None


- L2 loss를 구현하고, iteration마다 loss와 parameter가 어떻게 변하는 지 확인합니다.

In [8]:
for i in range(10):
  Y_hat = conv2d(X)
  l = (Y_hat - Y) **2
  conv2d.zero_grad() 
  l.sum().backward()
  conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
  print('iteration %d, loss %.3f' % (i+1, l.sum()))
  print('parameters:', conv2d.weight.data)

iteration 1, loss 16.062
parameters: tensor([[[[ 0.6616, -0.1601]]]])
iteration 2, loss 7.937
parameters: tensor([[[[ 0.4224, -0.8235]]]])
iteration 3, loss 4.120
parameters: tensor([[[[ 0.9192, -0.5982]]]])
iteration 4, loss 2.244
parameters: tensor([[[[ 0.7172, -0.9739]]]])
iteration 5, loss 1.275
parameters: tensor([[[[ 1.0039, -0.7985]]]])
iteration 6, loss 0.750
parameters: tensor([[[[ 0.8546, -1.0189]]]])
iteration 7, loss 0.453
parameters: tensor([[[[ 1.0252, -0.8938]]]])
iteration 8, loss 0.279
parameters: tensor([[[[ 0.9215, -1.0267]]]])
iteration 9, loss 0.174
parameters: tensor([[[[ 1.0255, -0.9414]]]])
iteration 10, loss 0.109
parameters: tensor([[[[ 0.9557, -1.0230]]]])


- 찾은 parameter가 우리가 정의했던 K와 비슷한 값을 가지는지 확인합니다.

In [9]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9557, -1.0230]])